In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModel, BitsAndBytesConfig
import pandas as pd
import numpy as np
import pickle
import tqdm
import torch
import os
import time
print(torch.__version__)

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print("device:", device)

2.1.1
device: cuda


In [2]:
model4bitconfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [5]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
print(f"Loading {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_fast = False,
                                          padding_side = "left",
                                          add_eos_token = True,
                                          add_bos_token = True,
                                         )
# tokenizer.add_special_tokens({"pad_token":"<pad>"})

max_memory_mapping = {0: "22GiB", 1: "10GiB", "cpu": "20GiB"}
# max_memory_mapping = {0: "10GiB", 1: "9GiB", 2: "9GiB", 3: "10GiB", "cpu":"20GiB"}
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map = 'auto',
                                             max_memory = max_memory_mapping,
                                             quantization_config = model4bitconfig).eval()
# model.resize_token_embeddings(len(tokenizer),pad_to_multiple_of=8)

Loading mistralai/Mixtral-8x7B-Instruct-v0.1...


ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [4]:
tokenizer.vocab_size

32000

In [9]:
def getZeroshotInference(model, content):
    prompts = content
    print(prompts)
    model_inputs = tokenizer(prompts,
                             truncation=True,
                             padding = True,
                             return_tensors="pt").to(device)
    
    outputs = model.generate(**model_inputs,
                             max_new_tokens=256,
                             do_sample = True,
                             temperature=0.01,
                             top_p=0.9
                            )
    
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(response)
    only_response = [response[i].strip()[len(prompt):] for i, prompt in enumerate(prompts)]
    print(only_response)
    # print("Response:", response[len(prompt):])
    # return response[len(prompt):]

In [10]:
content = [
    "How are you? What can you help me with?",
    "Show me a code to check if a number is pallindrome.",
    "How should I book a flight ticket to Alaska?",
    "Does God exist?",
    "How are you? What can you help me with?",
    "Show me a code to check if a number is pallindrome.",
    "How should I book a flight ticket to Alaska?",
    "Does God exist?"
]
import time
start = time.time()
getZeroshotInference(model, content)
print(time.time() - start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['How are you? What can you help me with?', 'Show me a code to check if a number is pallindrome.', 'How should I book a flight ticket to Alaska?', 'Does God exist?', 'How are you? What can you help me with?', 'Show me a code to check if a number is pallindrome.', 'How should I book a flight ticket to Alaska?', 'Does God exist?']


IndexError: piece id is out of range.